In [1]:
from abc import ABCMeta, abstractmethod
from queue import PriorityQueue
import random
import math

# Greedy Best-First Search implementation

<img src="greedy.png" width="400"/>

In [2]:
class Greedy():
    """
    Greedy Best-First Search implementation

    Methods
    -------
    search(state)
        Runs Greedy Best-First Search starting from a given start state, and returns the list of actions to reach a goal state and the number of expanded states.
    successor(state)
        Finds the list of successors for a given state.
    goal_test(state)
        Checks if the current state is a goal state.
    """

    __metaclass__ = ABCMeta

    def search(self, state):
        """
        Runs Greedy Best-First Search starting from a given start state, and returns the list of actions to reach a goal state and the number of expanded states.

        Parameters
        ----------
        state
            A tuple describing a unique world configuration.

        Returns
        -------
        list
            A list of actions that must be applied in sequence to the start state to reach the goal state, or None if no solution was found.
        int
            The number of states expanded during the search.
        """

        num_expanded_states = 0 # number of states expanded during the search
        expanded_states = []    # path from the start state to the current state
        visited_states = set()  # set of states in the path from the start state to the current state

        states_to_expand = PriorityQueue()                        # stack of states to be expanded
        states_to_expand.put((math.inf, 0, state, -1, None))
        while not states_to_expand.empty():
            heur_cost, path_cost, state, parent, action = states_to_expand.get() # get next state to be expanded

            if state in visited_states:                           # ignore cycles
                continue

            num_expanded_states += 1                              # add current state to the solution
            visited_states.add(state)
            expanded_states.append((parent, action))

            if self.goal_test(state):                             # if current state is a goal state, return solution
                solution = []
                while parent != -1:
                    solution.append(action)
                    parent, action = expanded_states[parent]
                solution.reverse()
                return solution, path_cost, num_expanded_states
            
            for action, child, act_cost, heur_cost in self.successor(state): # add successors to the stack of states to be expanded
                states_to_expand.put((heur_cost, act_cost+path_cost, child, num_expanded_states-1, action))

        return None, math.inf, num_expanded_states                # if no solution is found, return None

    @abstractmethod
    def successor(self, state):
        """
        Finds the list of successors for a given state.

        Parameters
        ----------
        state
            A tuple describing a unique world configuration.

        Returns
        -------
        list
            A list of pairs (action,state) with all states that can be reached from the given state with a single action.
        """
        pass

    @abstractmethod
    def goal_test(self, state):
        """
        Checks if the current state is a goal state.

        Parameters
        ----------
        state
            A tuple describing a unique world configuration.

        Returns
        -------
        bool
             True if the given state is a goal state, and False otherwise.
        """
        pass

# Traveling from Arad to Bucharest

- A graph of roads connecting different cities in Romania. The goal is to reach Bucharest (B) from Arad (A).

<img src="romania.png" width="600"/>

- **Heuristic function**: straight-line distance to Bucharest.

<img src="heuristic.png" width="400"/>

In [3]:
class RomaniaTravel(Greedy):
    """
    Traveling in Romania solution using Greedy Best-First Search.

    Methods
    -------
    show()
        Visualize the current state.
    move(action)
        Apply an action to the current state.
    successor(state)
        Finds the list of successors for a given state.
    goal_test(state)
        Checks if the current state is a goal state.
    """

    def __init__(self, state='Arad'):
        """
        Parameters
        ----------
        state
            A string describing a location. If None is provided, it is set to 'Arad'.
        """
        self.state = state
        self.roads = [
            ["Arad", "Zerind", 75],
            ["Arad", "Sibiu", 140],
            ["Arad", "Timisoara", 118],
            ["Zerind", "Oradea", 71],
            ["Oradea", "Sibiu", 151],
            ["Timisoara", "Lugoj", 111],
            ["Sibiu", "Fagaras", 99],
            ["Sibiu", "Rimnicu Vilcea", 80],
            ["Lugoj", "Mehadia", 70],
            ["Fagaras", "Bucharest", 211],
            ["Rimnicu Vilcea", "Pitesti", 97],
            ["Rimnicu Vilcea", "Craiova", 146],
            ["Mehadia", "Dobreta", 75],
            ["Bucharest", "Pitesti", 101],
            ["Bucharest", "Urziceni", 85],
            ["Bucharest", "Giurglu", 90],
            ["Pitesti", "Craiova", 138],
            ["Craiova", "Dobreta", 120],
            ["Urziceni", "Hirsova", 98],
            ["Urziceni", "Vaslui", 142],
            ["Hirsova", "Eforie", 86],
            ["Vaslui", "Lasi", 92],
            ["Lasi", "Neamt", 87]
        ]
        self.adj = self.__get_adjacency_list()
        self.heuristic = {
            "Arad": 366, "Bucharest": 0, "Craiova": 160, "Dobreta": 242, "Eforie": 161,
            "Fagaras": 176, "Giurgiu": 77, "Hirsowa": 151, "Lasi": 226, "Lugoj": 244,
            "Mehadia": 241, "Neamt": 234, "Oradea": 380, "Pitesti": 100, "Rimnicu Vilcea": 193,
            "Sibiu": 253, "Timisoara": 329, "Urziceni": 80, "Vaslui": 199, "Zerind": 374
        }
    
    def __get_adjacency_list(self):
        """
        Computes the adjacency list for the map of Romania.

        Return
        ----------
        dictionary
            A list per city with its neighbors and the distance to them.
        """
        adj = {}
        for x, y, c in self.roads:
            if x not in adj:
                adj[x] = []
            adj[x].append((y,c))
            if y not in adj:
                adj[y] = []
            adj[y].append((x,c))
        return adj

    def show(self):
        """
        Prints the current city.
        """
        print(self.state)

    def move(self, action):
        """
        Move to a different city using a road.self.state = b if self.state == a else a

        Parameters
        ----------
        action
            A pair of strings describing a source and a target location.
        """
        a, b = action
        self.state = a if a != self.state else b

    def successor(self, state):
        """
        Finds the list of successors for a given city.

        Parameters
        ----------
        state
            A string describing a location.

        Returns
        -------
        list
            A list of pairs (action,state,cost) with all cities that can be reached from the given city with a single move.
        """
        successors = [((state,child), child, cost, self.heuristic[child]) for child, cost in self.adj[state]]
        return successors

    def goal_test(self, state):
        """
        Checks if the current city is a goal state.

        Parameters
        ----------
        state
            A string describing a location.

        Returns
        -------
        bool
             True if the given state is a goal state, and False otherwise.
        """

        if state == 'Bucharest':
            return True
        else:
            return False

In [4]:
travel = RomaniaTravel()
print('Start state:')
travel.show()

actions, cost, num_states = travel.search(travel.state)
if actions is not None:
    print('Found a solution with cost {} after expanding {} states!'.format(cost, num_states))
    travel.show()
    for action in actions:
        print(action)
        travel.move(action)
        travel.show()
else:
    print('Could not find a solution after expanding {} states!'.format(num_states))

Start state:
Arad
Found a solution with cost 450 after expanding 4 states!
Arad
('Arad', 'Sibiu')
Sibiu
('Sibiu', 'Fagaras')
Fagaras
('Fagaras', 'Bucharest')
Bucharest
